In [3]:
import textwrap
from edsl.app import App, RankingApp
from edsl.app.output_formatter import OutputFormatter
from edsl import (
    Survey,
    QuestionFreeText,
    QuestionList,
    QuestionMultipleChoice,
    QuestionNumerical,
    Agent,
)

yc_advice = textwrap.dedent(
    """\
Your two sentence description is a concise explanation of your company for investors.
You'll practice using it during YC to introduce your company to other founders in the batch.

Your Group Partners will help you perfect your two sentence description.
A great two sentence description is a powerful tool that signals you're a clear thinker and succinct communicator.
This impresses investors and other useful people like potential hires or press.

Here are some examples of a good two sentence description:

    Stripe (S09) - Stripe is the easiest way for developers to process payments online. 
    In just 7 lines of code, you can start accepting payments from customers in 135+ currencies 
    without dealing with banks, compliance, or security headaches that normally take months to figure out.

    PostHog (W20) - PostHog is open source product analytics. Unlike closed-source alternatives that charge 
    $30K+/year and lock away your data, we give you complete control, unlimited events, and the ability to 
    deploy on your own infrastructure in under 5 minutes.

    Airbnb (W09) - Airbnb is a marketplace where you can book a home or apartment when traveling. 
    You get 3x more space for half the price of hotels, plus a kitchen, local neighborhood feel, and 
    unique stays like treehouses and castles that make your trip unforgettable.

A good two sentence description is memorable, creates a clear mental image of what your company does and 
makes the listener curious to learn more. Test it during the batch by asking a 
new founder each week to explain what your company does after hearing your two sentence description.

The three most common mistakes founders make when creating their two sentence pitch are making 
it too long, using too many buzzwords and failing to include traction.

Recommended structure

The first sentence plainly explains what the startup does. 
Anyone who hears it should have a clear sense for what it is used for, and ideally by whom. 
Nobody can engage with your startup if they don’t even understand what it is.

The second sentence plainly makes people understand why your startup is awesome and better than everything else. 
There are a bunch of ways to do this: 
specific user story that sounds way better, 
experience the founding team has that emphasizes they’re the ones to fix it, 
or even super fast growth during the YC batch.

It's not meant to be all-encompassing of the whole pitch

Think of your two sentence description like the trailer to a movie. 
It's goal is to pluck out just the most interesting things about your company 
to convince investors they should spend more time with you. Don’t try and cram everything 
about your company into it — investors are impressed by clear and concise communication, 
and the two sentence pitch is the appetizer not the main course.

Use plain English

Sometimes founders try and impress investors by stringing together popular buzzwords they think investors want to hear. 
It usually sounds something like this: 
<bad_example>
“QuantumAI is an AI-powered platform revolutionizing the 
future of business by streamlining complex workflows to drive efficiency for our customers.” 
</bad_example>
Don’t do this. Instead use informal, simple language your mother would understand.
"""
).strip()

# Initial survey to gather startup information
initial_survey = Survey(
    [
        QuestionFreeText(
            question_name="startup_name", question_text="What is your startup's name?"
        ),
        QuestionFreeText(
            question_name="startup_does", question_text="What does your startup do?"
        ),
        QuestionFreeText(
            question_name="traction", question_text="What traction do you have?"
        ),
        QuestionFreeText(
            question_name="bio",
            question_text="Tell me your bio (or your founding team's bio)",
        ),
        QuestionNumerical(
            question_name="n", question_text="How many samples do you want to generate?"
        ),
    ]
)


criteria = """
<important>
First sentence is what the startup does.
Second sentence is impressive traction or background of founders
</important>

A few key points:
- These 2 sentences are for investors, peers and press, not end users.
- It should invite further questions
- Don't lead with the problem
- All else equal, shorter is better
- Sharp and crisp.
- Every non-essential word hurts.
- Think movie trailer.

If traction is impressive, lead with that.
- Real revenue
- Booked LOIs
- Growth

Traction Examples
- "Over $100K in ARR"
- Contracts signed with Cedar-Sinai Medical center
- Sharp and short

If bio is impressive and important, that can be a good second sentence:
- Serial entrepreneur with 2 exits
- Founding engineer at Uber
- PhD from Stanford in molecular biology

Things that get investors excited:
- Signs there is a large market
- Great founder-market fit
- Signals of traction
"""

# YC partner agent with expertise in crafting 2-sentence descriptions
yc_partner = Agent(
    name="yc_partner",
    traits={
        "persona": textwrap.dedent(
            """\
You are a seasoned partner at Y Combinator very skilled at helping startups
come up with a 2 sentence description.
"""
            + yc_advice
            + criteria
        )
    },
)

# Question to generate candidate 2-sentence descriptions
q_generate_candidates = QuestionList(
    question_name="two_sentences",
    question_text=textwrap.dedent(
        """
Based on this information:

Name: {{ scenario.startup_name }},
What they do: {{ scenario.startup_does}},
Traction: {{ scenario.traction }},
Founder bio: {{ scenario.bio }}

Come up with at least {{ scenario.n }} candidate variations for the YC 2 sentence description.
It is fine to omit aspects of the startup that you know - it's meant to be a short pitch.
Make them different from each other to create a diverse set of candidates.
Variation can be induced by highlighting different aspects of the startup, or using a different tone/style/wording.
"""
    ),
)

# Question to shorten the candidates
q_shorten = QuestionFreeText(
    question_name="shorten",
    question_text="Please make this even more concise and sharper, aiming for 1/2 the length: {{ scenario.two_sentences }}",
)

# Stage 1: Generate and shorten candidates
generate_job = (
    Survey([q_generate_candidates])
    .by(yc_partner)
    .select("answer.two_sentences", "scenario.*")
    .to_scenario_list()
    .expand("two_sentences")
    .to(Survey([q_shorten]).by(yc_partner))
)

# Output formatter for Stage 1
candidates_formatter = (
    OutputFormatter(name="Candidate Pitches")
    .select("answer.shorten")
    .rename({"answer.shorten": "two_sentences"})
    .to_scenario_list()
)

# Stage 1 App: Generate candidates
generation_app = App(
    application_name="YC Two Sentence Generator",
    description="Generates candidate 2-sentence startup descriptions using YC best practices",
    initial_survey=initial_survey,
    jobs_object=generate_job,
    output_formatters=candidates_formatter,
)

In [3]:
candidates = generation_app.output(
    params={
        "startup_name": "Expected Parrot",
        "startup_does": """Builds open-source tools for using AI for generating AI personas that users can then
use to explore scenarios. There is a Python library, EDSL, where users can specify Agents, Scenarios and Survey Questions.
It works with lots of different models.
Companies have used it for pricing, and scenario planning. 
They ask questions to simulated customers that are otherwise hard to ask regular customers.
They are mostly simulating their customers and running experiments on those digital twins.
Academics are seeing if they can replicate existing social science experiments.
Some customers are using it as a kind of data labeling tool, using features like piping 
and skip-logic to ask follow-up questions based on answers.""",
        "traction": "Fast growing usage among academics; $120K in enterprise contracts",
        "bio": "Ex-Uber; MIT Professor who pioneered this approach.",
        "n": 20,
    },
    verbose=True,
)
candidates

,two_sentences
0,"Expected Parrot offers open-source AI tools to simulate customer interactions. Boasting $120K in enterprise contracts and led by an ex-Uber MIT professor, we're rapidly expanding in academia and business."
1,"Expected Parrot creates open-source AI tools for scenario planning and customer simulation. Founded by an MIT professor and former Uber engineer, we've secured $120K in enterprise contracts."
2,"Expected Parrot provides a Python library for AI persona generation to test digital twins. Founded by an MIT professor and ex-Uber engineer, we've secured $120K in enterprise contracts."
3,"Expected Parrot's open-source tools simulate customer scenarios for pricing and strategy testing. With $120K in enterprise contracts and the founder's MIT and Uber background, we're rapidly gaining traction in academia."
4,"Expected Parrot offers AI tools for simulating customer interactions, simplifying complex scenarios. Founded by an MIT professor and ex-Uber engineer, we've secured $120K in enterprise contracts."
5,"Expected Parrot's platform lets businesses simulate customer interactions with AI personas. Led by an MIT professor and ex-Uber engineer, we've secured $120K in enterprise contracts."
6,"Expected Parrot provides a Python library for AI personas to simulate customer scenarios. We've secured $120K in enterprise contracts, led by our MIT professor founder."
7,"Expected Parrot creates open-source tools for AI persona generation. We have $120K in enterprise contracts, led by an MIT professor and ex-Uber engineer."
8,"Expected Parrot offers AI tools to simulate customer interactions for businesses. Founded by an MIT professor and ex-Uber engineer, we've secured $120K in enterprise contracts."
9,"Expected Parrot provides a Python library for AI persona creation and scenario simulations. We've secured $120K in enterprise contracts, led by an MIT professor and former Uber engineer."


In [4]:
from edsl.app import AppRegistry

AppRegistry.from_package("edsl.app.examples")

['advice_to_checklist',
 'agent_blueprint_creator',
 'auto_survey',
 'Typo Checker',
 'color_survey',
 'Conjoint Analysis Setup',
 'Conjoint Profile Generator',
 'create_personas',
 'data_labeling',
 'Eligible Agents',
 'Food Health Ranking',
 'Food Health TrueSkill Ranking',
 'jeopardy',
 'Meal Planner',
 'packing_list',
 'panel_reaction',
 'referee_report',
 'Robot VC',
 'rubric_generator',
 'story_time',
 'Survey Option Inference',
 'variant_creator']

In [9]:
survey = AppRegistry.get_app("advice_to_checklist")(advice_text=yc_advice)

In [10]:
survey

,question_name,question_text,question_options,question_type
0,question_0,Does the first sentence clearly explain what the startup does?: {{ scenario.item }},"['No', 'Yes']",yes_no
1,question_1,Is the target user or customer clearly identified in the first sentence?: {{ scenario.item }},"['No', 'Yes']",yes_no
2,question_2,Does the second sentence explain why the startup is awesome or better than competitors?: {{ scenario.item }},"['No', 'Yes']",yes_no
3,question_3,Is the language used simple and free of buzzwords?: {{ scenario.item }},"['No', 'Yes']",yes_no
4,question_4,Is the description concise and limited to two sentences?: {{ scenario.item }},"['No', 'Yes']",yes_no
5,question_5,Does the description avoid trying to include everything about the company?: {{ scenario.item }},"['No', 'Yes']",yes_no
6,question_6,Does the description make the listener curious to learn more?: {{ scenario.item }},"['No', 'Yes']",yes_no
7,question_7,Is there a specific user story or unique selling point included?: {{ scenario.item }},"['No', 'Yes']",yes_no
8,question_8,Is the description memorable and easy to repeat?: {{ scenario.item }},"['No', 'Yes']",yes_no
9,question_9,Has the description been tested with new founders for clarity and understanding?: {{ scenario.item }},"['No', 'Yes']",yes_no


In [11]:
r = survey(item="Expect Parrot is Bringing About the Quantum Singularity")
r[0].q_and_a()
r[0].q_and_a().tally("answer")

In [12]:
r[0].q_and_a()

,question_text,answer
0,Does the first sentence clearly explain what the startup does?: {{ scenario.item }},No
1,Is the target user or customer clearly identified in the first sentence?: {{ scenario.item }},No
2,Does the second sentence explain why the startup is awesome or better than competitors?: {{ scenario.item }},No
3,Is the language used simple and free of buzzwords?: {{ scenario.item }},No
4,Is the description concise and limited to two sentences?: {{ scenario.item }},No
5,Does the description avoid trying to include everything about the company?: {{ scenario.item }},Yes
6,Does the description make the listener curious to learn more?: {{ scenario.item }},Yes
7,Is there a specific user story or unique selling point included?: {{ scenario.item }},No
8,Is the description memorable and easy to repeat?: {{ scenario.item }},No
9,Has the description been tested with new founders for clarity and understanding?: {{ scenario.item }},No


In [13]:
r[0].q_and_a().tally("answer")

,answer,count
0,No,8
1,Yes,2


In [2]:
AppRegistry.list()

,name,description
0,agent_blueprint_creator,Creates an agent blueprint for generating personas from a population description
1,auto_survey,Automatically generate a survey based on the user's input.
2,Typo Checker,Check for typos in the question text
3,color_survey,Create a customized survey about favorite colors with insights and analysis
4,Conjoint Analysis Setup,"Analyzes a product to identify components, potential levels, and current levels for conjoint analysis"
5,Conjoint Profile Generator,Generates random product profiles for conjoint analysis from attribute definitions (designed for app chaining with >>)
6,create_personas,A persona generator.
7,data_labeling,A data labeling app.
8,Eligible Agents,This is a constructed agent. It is randomly collected from a population. Is the agent eligible to participate in the study?
9,Food Health Ranking,Ranks foods from healthiest to least healthy using pairwise comparisons.


In [7]:
AppRegistry.get_app("rubric_generator").output(
    params={"artifact_description": "2 sentence YC startup pitch"}
)

,question_name,question_text,question_options,option_labels,question_type
0,question_0,Clarity of Problem Statement: {{ scenario.item}},"[1, 2, 3, 4, 5]","{1: 'The problem statement is vague and confusing, lacking any clear focus.', 2: 'The problem statement is somewhat understandable but lacks specificity and clarity.', 3: 'The problem statement is clear and identifies a problem, but could be more concise.', 4: 'The problem statement is clear, concise, and identifies a relevant problem.', 5: 'The problem statement is exceptionally clear, concise, and precisely identifies a critical problem.'}",linear_scale
1,question_1,Innovativeness of Solution: {{ scenario.item}},"[1, 2, 3, 4, 5]","{1: '1: The solution lacks originality and is a direct copy of existing ideas.', 2: '2: The solution shows minimal innovation and only slightly improves upon existing ideas.', 3: '3: The solution is moderately innovative and offers a fresh perspective on existing solutions.', 4: '4: The solution is innovative and introduces a novel approach to a common problem.', 5: '5: The solution is highly innovative and groundbreaking, offering a completely new way to address a problem.'}",linear_scale
2,question_2,Market Potential: {{ scenario.item}},"[1, 2, 3, 4, 5]","{1: '1: The market potential is minimal or non-existent, with limited growth opportunities.', 2: '2: The market potential is low, with some growth opportunities but significant challenges.', 3: '3: The market potential is moderate, with reasonable growth opportunities and some challenges.', 4: '4: The market potential is strong, with good growth opportunities and manageable challenges.', 5: '5: The market potential is exceptional, with vast growth opportunities and minimal challenges.'}",linear_scale
3,question_3,Conciseness and Impact: {{ scenario.item}},"[1, 2, 3, 4, 5]","{1: '1: The pitch is overly wordy and lacks impact, failing to convey a clear message.', 2: '2: The pitch is somewhat concise but lacks a strong impact or clear focus.', 3: '3: The pitch is concise and conveys a basic level of impact.', 4: '4: The pitch is concise and impactful, effectively conveying its message.', 5: '5: The pitch is exceptionally concise and impactful, delivering a compelling message with precision.'}",linear_scale
4,question_4,Founder's Credibility: {{ scenario.item}},"[1, 2, 3, 4, 5]","{1: '1: The founder lacks relevant experience or credentials, raising doubts about their ability to execute the idea.', 2: '2: The founder has limited relevant experience or credentials, leading to some concerns about their capability.', 3: '3: The founder has moderate relevant experience or credentials, providing a reasonable level of confidence.', 4: '4: The founder has strong relevant experience or credentials, inspiring confidence in their ability to execute the idea.', 5: '5: The founder has exceptional relevant experience or credentials, greatly enhancing their credibility and instilling high confidence.'}",linear_scale


In [4]:
from edsl.app.examples.rubric_generator import app

In [5]:
AppRegistry.list()

,name,description
0,rubric_generator,A rubric generator.


In [ ]:
q_rank = QuestionMultipleChoice(
    question_name="better_pitch",
    question_text="Which of these two startup pitches is more compelling and effective for investors?"
    + criteria,
    question_options=[
        "{{ scenario.two_sentences_1 }}",
        "{{ scenario.two_sentences_2 }}",
    ],
    use_code=False,
)

ranking_app = RankingApp(
    ranking_question=q_rank,
    option_fields=["two_sentences_1", "two_sentences_2"],
    application_name="YC Two Sentence Ranker",
    description="Ranks 2-sentence startup descriptions by effectiveness",
    option_base="two_sentences",
    rank_field="pitch_rank",
)

sampled_candidates = candidates.sample(15)

# Stage 2: Rank the candidates
print("\n=== Stage 2: Ranking candidates ===")
ranked = ranking_app.output(params={"input_items": sampled_candidates}, verbose=True)
print(ranked.table())